In [29]:
# set up script
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import matplotlib as mpl
mpl.use('pgf')
plt.close('all')
from math import sqrt
default_width = 2.8 # in inches
default_ratio = (sqrt(5.0) - 1.0) / 2.0 # golden mean
mpl.rcParams.update({
    "text.usetex": True,
    "pgf.texsystem": "pdflatex",
    "pgf.rcfonts": False,
    "font.family": "serif",
    "font.serif": [],
    "font.size": 8,
    "font.sans-serif": [],
    "font.monospace": [],
    "figure.figsize": [default_width, default_width * default_ratio],
    "pgf.preamble": [
        # put LaTeX preamble declarations here
        r"\usepackage[utf8x]{inputenc}",
        r"\usepackage[T1]{fontenc}",
        # macros defined here will be available in plots, e.g.:
        r"\newcommand{\vect}[1]{#1}",
        # You can use dummy implementations, since you LaTeX document
        # will render these properly, anyway.
    ],
})

filename = 'Aachen'

C:\Users\Daniel\AppData\Local\Programs\Python\Python39\lib\_collections_abc.py:856: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


In [30]:
# load data
col = {'Id': {'is_parameter': False, 'is_lane_parameter': False, 'is_integer': True,
              'values': []},
       'Lane_Id': {'is_parameter': False, 'is_lane_parameter': True, 'is_integer': True,
                   'values': []},
       'Type': {'is_parameter': False, 'is_lane_parameter': False, 'is_integer': True,
                'values': [0, 2, 3]},
       'Traffic_signals': {'is_parameter': True, 'is_lane_parameter': False, 'is_integer': True,
                           'values': [0, 1]},
       'Index_link': {'is_parameter': False, 'is_lane_parameter': False, 'is_integer': True,
                      'values': [0, 1, 2]},
       'Is_built_up_area_road': {'is_parameter': True, 'is_lane_parameter': False, 'is_integer': True,
                                 'values': [0, 1]},
       'Incoming_lanes': {'is_parameter': True, 'is_lane_parameter': False, 'is_integer': True,
                          'values': np.linspace(0, 10, 11, dtype=int)},
       'Link_length': {'is_parameter': True, 'is_lane_parameter': False, 'is_integer': False,
                       'values': np.linspace(0, 800, 201, dtype=int)},
       'Intersection_area': {'is_parameter': True, 'is_lane_parameter': False, 'is_integer': False,
                             'values': np.linspace(0, 50, 51, dtype=int)},
       'Intersecting_angle': {'is_parameter': True, 'is_lane_parameter': False, 'is_integer': False,
                              'values': np.linspace(0, 200, 100, dtype=int)},
       'Lane_count': {'is_parameter': True, 'is_lane_parameter': False, 'is_integer': True,
                      'values': np.linspace(0, 10, 11, dtype=int)},
       'Next_type': {'is_parameter': True, 'is_lane_parameter': False, 'is_integer': True,
                     'values': [0, 2, 3, 4]},
       'Speed_limit': {'is_parameter': True, 'is_lane_parameter': False, 'is_integer': False,
                       'values': np.linspace(0, 140, 141, dtype=int)},
       'Lane_start_width': {'is_parameter': True, 'is_lane_parameter': True, 'is_integer': False,
                            'values': np.linspace(0, 400, 101, dtype=int)},
       'Lane_end_width': {'is_parameter': True, 'is_lane_parameter': True, 'is_integer': False,
                          'values': np.linspace(100, 400, 101, dtype=int)},
       'Lane_mean_width': {'is_parameter': True, 'is_lane_parameter': True, 'is_integer': False,
                           'values': np.linspace(100, 400, 101, dtype=int)},
       'Lane_max_curvature': {'is_parameter': True, 'is_lane_parameter': True, 'is_integer': False,
                              'values': np.linspace(0, 1.5, 61, dtype=float)},
       'Lane_heading': {'is_parameter': True, 'is_lane_parameter': True, 'is_integer': True,
                        'values': [-1, 0, 1]}
       }

tab10 = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray',
         'tab:olive', 'tab:cyan']
#table = pd.read_pickle(filename + '_stats.pkl')
table = pd.read_pickle('Berlin_df.pkl')
tableAC = pd.read_pickle('Aachen_df.pkl')
test = table[['Id', 'Type']].drop_duplicates(keep='first')
print('junctions:\t%s' %(test.shape,))
print('X-junctions:\t%s' %(test[test['Type'] == 2].shape,))
print('T-junctions:\t%s' %(test[test['Type'] == 0].shape,))

junctions:	(6629, 2)
X-junctions:	(1379, 2)
T-junctions:	(5131, 2)


In [31]:
# Left and right turning lanes on t-junctions
df1 = table

left_df = df1[(df1['Lane_heading'] == -1) & (df1['Lane_max_curvature'] > 0.01)]# & (df1['Type'] == 0)]
right_df = df1[(df1['Lane_heading'] == 1) & (df1['Lane_max_curvature'] > 0.01)]# & (df1['Type'] == 0)]
print('left turns:\t%s' %(left_df.shape,))
print('right turns:\t%s' %(right_df.shape,))
bins = np.linspace(0, 1.2, 61, dtype=float)
fig, axes = plt.subplots()
sns.histplot(ax=axes, x='Lane_max_curvature', data=left_df, bins=bins, stat='density', color='tab:olive', discrete=False, kde=True)
sns.histplot(ax=axes, x='Lane_max_curvature', data=right_df, bins=bins, stat='density', color='tab:blue', discrete=False, kde=True)
labels= ["left turn","right turn"]
plt.legend(labels)
axes.set_xlim(0,1.2)
plt.tight_layout()
plt.savefig('turning_curvature.pgf')
plt.close()


left turns:	(12992, 19)
right turns:	(14482, 19)


In [32]:
# intersection area
link_col = [field for field in col if not col[field]['is_lane_parameter']]
df1 = table[link_col]
df1 = df1.drop_duplicates(keep='first')

bins = col['Intersection_area']['values']
print('junction arms:\t%s' %(df1.shape,))
fig, axes = plt.subplots()
bins = np.linspace(0,25, 61, dtype=float)
sns.histplot(ax=axes, x='Intersection_area', data=df1, bins=bins, stat='probability', discrete=False, kde=True)
axes.set_xlim(0,25)
plt.tight_layout()
plt.savefig('intersection_area.pgf')

junction arms:	(19506, 12)


In [33]:
# mean lane width in junction
df1 = table
df1 = df1.drop_duplicates(keep='first')
dfac = tableAC.drop_duplicates(keep='first')

print(df1.shape)
fig, axes = plt.subplots()
bins = np.linspace(250, 450, 61, dtype=float)
sns.histplot(ax=axes, x='Lane_mean_width', data=df1, bins=bins, stat='probability', discrete=False, kde=True)
#sns.histplot(ax=axes, x='Lane_mean_width', data=dfac, bins=bins, stat='density', discrete=False, kde=True)
meanWidth = df1['Lane_mean_width'].mean()
meanWidthAc = dfac['Lane_mean_width'].mean()
plt.legend(["$\mu$ = " + "{:.0f}".format(meanWidth) + " $cm$"])
axes.set_xlim(250,450)
plt.tight_layout()
plt.savefig('lane_width.pgf')
plt.close()
print('Mean width B:\t%s' %(meanWidth))
print('Mean width AC:\t%s' %(meanWidthAc))

(44265, 19)
Mean width B:	365.65869276613245
Mean width AC:	353.80374019181454


In [34]:
# intersecting angle for x-junciton

link_col = [field for field in col if not col[field]['is_lane_parameter']]
df1 = table[link_col]
df1 = df1.drop_duplicates(keep='first')

bins = col['Intersecting_angle']['values']
df1 = df1[df1['Type'] == 2]
print(df1.shape)

fig, axes = plt.subplots()
bins = np.linspace(50, 125, 61, dtype=float)
sns.histplot(ax=axes, x='Intersecting_angle', data=df1, bins=bins, stat='probability', discrete=False, kde=True)
#sns.kdeplot(ax=axes, x='Intersecting_angle', data=df1)
meanWidth = df1['Intersecting_angle'].mean()
plt.legend(["$\mu$ = " + "{:.2f}".format(meanWidth) + " $deg$"])
axes.set_xlim(50,125)
plt.tight_layout()
plt.savefig('intersecting_angle_X.pgf')
plt.close()

(5357, 12)


In [35]:
# intersecting angle for t-junciton

link_col = [field for field in col if not col[field]['is_lane_parameter']]
df1 = table[link_col]
df1 = df1.drop_duplicates(keep='first')

bins = col['Intersecting_angle']['values']
df1 = df1[df1['Type'] == 0]
print(df1.shape)

fig, axes = plt.subplots()
bins = np.linspace(25, 210, 61, dtype=float)
sns.histplot(ax=axes, x='Intersecting_angle', data=df1, bins=bins, stat='probability', discrete=False, kde=True)
#sns.kdeplot(ax=axes, x='Intersecting_angle', data=df1)
axes.set_xlim(25,210)
plt.tight_layout()
plt.savefig('intersecting_angle_T.pgf')
plt.close()

(13745, 12)


In [36]:
#### stat parameter ####

# To work on parameters specific to the links we need first to get rid of the lane parameters
# and then get rid of the duplicates
link_col = [field for field in col if not col[field]['is_lane_parameter']]
df1 = table[link_col]
df1 = df1.drop_duplicates(keep='first')

bins = col['Intersecting_angle']['values']

fig, axes = plt.subplots(1, 4, sharex=False, sharey=False, figsize=(20, 10))

# count shows the number of observations
sns.histplot(ax=axes[0], data=df1, x='Intersecting_angle', bins=bins, stat='count', discrete=False, color=tab10[0], kde=True)
# frequency shows the number of observations divided by the bin width
sns.histplot(ax=axes[1], data=df1, x='Intersecting_angle', bins=bins, stat='frequency', discrete=False, color=tab10[1], kde=True)
# density normalizes counts so that the area of the histogram is 1
sns.histplot(ax=axes[2], data=df1, x='Intersecting_angle', bins=bins, stat='density', discrete=False, color=tab10[2], kde=True)
# probability normalizes counts so that the sum of the bar heights is 1
sns.histplot(ax=axes[3], data=df1, x='Intersecting_angle', bins=bins, stat='probability', discrete=False, color=tab10[3], kde=True)

# kde keyword : compute a kernel density estimate to smooth the distribution
# and show on the plot as (one or more) line(s)

axes[0].set_xlim(bins[0] - 1, bins[-1] + 1)
axes[1].set_xlim(bins[0] - 1, bins[-1] + 1)
axes[2].set_xlim(bins[0] - 1, bins[-1] + 1)
axes[3].set_xlim(bins[0] - 1, bins[-1] + 1)

fig.suptitle('stat parameter : count, frequency, density, probability')
plt.close()

In [37]:
df2 = df1[df1['Type'] == 0]
bins = col['Lane_count']['values']

fig, axes = plt.subplots(1, 2, sharex=False, sharey=False, figsize=(20, 10))

sns.histplot(ax=axes[1], data=df2, x='Lane_count', bins=bins, stat='density', discrete=True,
             color=tab10[-1], shrink=0.75)

sns.histplot(ax=axes[0], data=df2, x='Lane_count', hue='Index_link', bins=bins, stat='density',
             common_norm=False,
             discrete=True, multiple='dodge', shrink=0.5, palette='tab10')

axes[0].set_xlim(bins[0] - 1, bins[-1] + 1)
axes[1].set_xlim(bins[0] - 1, bins[-1] + 1)
# set_xticks sets the values displayed on the x-axis
axes[0].set_xticks(bins)
axes[1].set_xticks(bins)
fig.suptitle('hue parameter : Index_link ')
plt.show()
plt.close()


<ipython-input-37-dc70751dfb0d>:19: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
